# Ex 1: Conventional MIMO channel

You can use the following header to load the packages needed. 

In [182]:
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy.linalg import qr
from numpy.linalg import inv, norm
from scipy.stats import norm as normpdf

### Implement the zero-forcing (ZF) detector

In [22]:
# =============================================================================
# Zero-Forcing (ZF) Detector
# =============================================================================
def zf_detector(y, H, constellation):
    """
    Zero-forcing detector:
      - Compute LS solution: x_ls = (H^H H)^{-1} H^H y.
      - Then round each entry to the closest constellation point.
      
    Parameters:
      y : Received vector (complex, shape (nr,))
      H : Channel matrix (complex, shape (nr, nt))
      constellation : 1D numpy array of constellation points
      
    Returns:
      x_hat : Detected transmit vector (complex, shape (nt,))
    """
    x_ls = """"""""
    
    # Round each entry to the nearest constellation point
    x_hat = np.array([constellation[np.argmin(np.abs(x - constellation))]
                      for x in x_ls])
    
    return x_hat


### Implement the sphere decoder (SD)

In [91]:
# =============================================================================
# Sphere-Decoding (SD) Detector
# =============================================================================
def sphere_decoding(y, H, constellation, rho=None):
    """
    Sphere Decoding detector:
      Preprocess:
        1. Compute a permutation matrix P that reverses the order of entries.
        2. Compute the QR decomposition of H @ P, i.e.  Q,R such that H P = Q R.
        3. Form L = P R P and s = P (Q^H y). Then, ||y - H x|| = || s - L x||.
      
      The algorithm then performs a recursive search over the tree of possible
      x values (with x drawn from the finite constellation) while pruning paths
      whose partial metric exceeds the current radius.
      
    Parameters:
      y : Received vector (complex, shape (nr,))
      H : Channel matrix (complex, shape (nr, nt))
      constellation : 1D numpy array of constellation points (real)
      rho : (Optional) initial radius. If not provided, it is set to the squared
            metric of the ZF solution.
            
    Returns:
      best_solution : Detected transmit vector (complex, shape (nt,))
      best_metric   : Squared Euclidean metric value of the detected solution.
    """
    nt = H.shape[1]
    
    # Permutation matrix that reverses the order
    P = np.flip(np.eye(nt), axis=0)
    # QR decomposition of H @ P
    #    Q, R = qr(H @ P, mode='economic')
    Q, R = qr(np.flip(H, axis=1), mode='economic')
    
    # Preprocessing: L = P R P, s = P (Q^H y)
    L = P @ R @ P
    #L = np.flip(np.flip(R, axis=0), axis=1)
    #    s = P @ (Q.conj().T @ y)
    s = np.flip(Q.conj().T @ y, axis=0)
    
    # Define the incremental metric at level k (0-indexed)
    def metric_increment(k, x_vec):
        
        # For level k, compute:
        #   alpha = L[k,k] * x_vec[k] - ( s[k] - sum_{j=0}^{k-1} L[k,j]*x_vec[j] )
        if k == 0:
            return np.abs(L[0,0]*x_vec[0] - s[0])**2
        else:
            sum_prev = np.dot(L[k, :k], x_vec[:k])
            alpha = L[k, k]*x_vec[k] - (s[k] - sum_prev)
            return np.abs(alpha)**2
        
        #return np.abs(np.dot(L[k, :(k+1)], x_vec[:(k+1)]) - s[k])**2
        
    best_metric = np.inf
    best_solution = None
    x_candidate = np.zeros(nt, dtype=H.dtype)
    
    # If no initial radius is provided, initialize using the ZF detector.
    if rho is None:
        x_zf = """"""
        best_metric = """"""
        best_solution = x_zf.copy()
    else:
        best_metric = rho**2

    # Recursive sphere decoding search
    def rec_sd(k, current_metric):
        nonlocal best_metric, best_solution, x_candidate
        if k == nt:
            # Full candidate reached; update best if necessary
            if current_metric < best_metric:
                best_metric = current_metric
                best_solution = x_candidate.copy()
            return
        # At level k, use a zero-forcing estimate at that level as a guide:
        if np.abs(L[k,k]) > 1e-12:
            zf_level = s[k] / L[k,k]
        else:
            zf_level = 0
        # Sort constellation points by proximity to zf_level
        cand_sorted = sorted(constellation, key=lambda c: np.abs(c - zf_level))
        for cand in cand_sorted:
            x_candidate[k] = cand
            inc = metric_increment(k, x_candidate)
            new_metric = current_metric + inc
            if new_metric < best_metric:
                rec_sd(k+1, new_metric)

    rec_sd(0, 0)
    return best_solution, best_metric

### Run the simulation 

Generate a M^2-QAM constellation (first generate a PAM constellation) 

In [107]:
def generate_QAM_constellation(M):
    PAM_M = """""""
    constellation = []
    for a in PAM_M:
        for b in PAM_M:
            constellation.append(a + 1j*b)
    return np.array(constellation)

Configurations of the simulation 

In [112]:
# For reproducibility / fairness of comparison
np.random.seed(0)

# System dimensions
nt = 4       # number of transmit antennas
nr = 4       # number of receive antennas (assume nr >= nt, not needed for SD)

# Constellation size is M^2
M = 4
constellation = generate_QAM_constellation(M)
    
# SNR values in dB (for 1/σ²): 
snr_dB_vals = np.arange(0, 15, 3)
N = 1000  # Number of Monte Carlo trials per SNR

In [ ]:
###############################################
# Simulation: Compare ZF and SD for QAM
###############################################
error_zf = np.zeros(len(snr_dB_vals))
error_sd = np.zeros(len(snr_dB_vals))
runtime_zf = np.zeros(len(snr_dB_vals))
runtime_sd = np.zeros(len(snr_dB_vals))

for idx, snr_db in enumerate(snr_dB_vals):
    # SNR_linear = 10^(snr_db/10), noise variance sigma² = 1/SNR_linear.
    snr_linear = 10**(snr_db/10)
    sigma2 = 1.0 / snr_linear

    errors_zf = 0
    errors_sd = 0
    total_time_zf = 0
    total_time_sd = 0

    for trial in range(N):
        # Generate random complex channel H (CN(0,1) entries)
        H = """"""
        # Generate transmitted vector x_true from 16-QAM constellation
        x_true = np.random.choice(constellation, size=(nt,))
        # Generate noise z ~ CN(0, sigma² I)
        z = """"""
        # Received vector: y = H x_true + z
        y = """"""
        
        # Zero-Forcing Detector
        t0 = time.time()
        x_hat_zf = """"""
        total_time_zf += time.time() - t0
        if not np.array_equal(x_hat_zf, x_true):
            errors_zf += 1

        # Sphere-Decoding Detector
        t0 = time.time()
        x_hat_sd, _ = """"""
        total_time_sd += time.time() - t0
        if not np.array_equal(x_hat_sd, x_true):
            errors_sd += 1

    error_zf[idx] = errors_zf / N
    error_sd[idx] = errors_sd / N
    runtime_zf[idx] = total_time_zf / N
    runtime_sd[idx] = total_time_sd / N

    print("SNR = {:.1f} dB: ZF error = {:.4f}, SD error = {:.4f}".format(
        snr_db, error_zf[idx], error_sd[idx]))
    print("          Average runtime: ZF = {:.6f} s, SD = {:.6f} s".format(
        runtime_zf[idx], runtime_sd[idx]))

# Plot average vector detection error (semilogy)
plt.figure()
plt.semilogy("""""")
plt.semilogy("""""")
plt.xlabel('1/σ² (dB)')
plt.ylabel('Average Vector Detection Error')
plt.title('Detection Error vs SNR')
plt.legend()
plt.grid(True, which='both')

# Plot average runtime vs SNR
plt.figure()
plt.semilogy("""""")
plt.semilogy("""""")
plt.xlabel('1/σ² (dB)')
plt.ylabel('Average Runtime (s)')
plt.title('Runtime vs SNR')
plt.legend()
plt.grid(True)

plt.show()


# Ex2: MIMO with one-bit ADC

Now, let us consider the case with 
$$\mathbf{y} = \mathrm{sign}(\mathbf{H} \mathbf{x} + \mathbf{z})$$
where $\mathbf{z}$ is i.i.d. $\mathcal{N}(0,\sigma^2)$ and $\mathbf{x}$ is a vector of $M$-PAM symbols. 

You may need the following headers. 

In [175]:
import numpy as np
import time
import matplotlib.pyplot as plt
from numpy.linalg import norm, inv, cholesky
from scipy.optimize import minimize
from scipy.stats import norm as normpdf

### Define the objective function, gradient, and Hessian

In [176]:
###############################################
# One-Bit ADC Negative Log-Likelihood and Derivatives
###############################################

def Q_func(u):
    # Q(u) = 1 - Phi(u)
    return normpdf.sf(u)

def f_func(x, H, y, sigma):
    # x: real vector (n_t,), H: (n_r x n_t), y: (n_r,) with y_i in {-1,1}, sigma: positive real
    # f(x) = - sum_{i=1}^{n_r} log( Q( - y_i * (h_i^T x) / sigma ) )
    # where h_i^T is the i-th row of H.

    f = """"""""
    return f

def grad_f(x, H, y, sigma):
    # Gradient: ∇f(x) = - sum_{i=1}^{n_r} y_i/sigma * (phi(u_i)/Q(u_i)) * h_i
    
    """"""""
    grad = """"""
    return grad

def hess_f(x, H, y, sigma):
    # Hessian: Hess f(x) = sum_{i=1}^{n_r} [ phi(u_i)/(Q(u_i))^2 * (phi(u_i)-u_i*Q(u_i)) ] * h_i * h_i^T / sigma^2
    u = """""" 
    factors = """"""
    n_r = H.shape[0]
    n_t = H.shape[1]
    Hhess = np.zeros((n_t, n_t))
    for i in range(n_r):
        hi = H[i, :].reshape(-1, 1)
        Hhess += factors[i] * (hi @ hi.T) / (sigma**2)
    return Hhess

### Zero-forcing decoder

In [177]:
def round_to_constellation(x, constellation):
    """"""""
    return x_hat

def ZF_detector_onebit(y, H, sigma, constellation):
    """
    One-bit ADC "zero-forcing" detection:
      1. Find the stationary point x_S that minimizes f(x) using BFGS.
      2. Then round x_S componentwise to the nearest constellation point.
    """
    n_t = H.shape[1]
    # Use a random initial guess (e.g., zero vector)
    x0 = np.zeros(n_t)
    res = minimize(f_func, x0, args=(H, y, sigma), method='BFGS', jac=grad_f, options={'gtol':1e-6, 'disp':False})
    x_S = res.x
    # Zero-forcing detection: round the stationary point to the constellation.
    x_ZF = """"""
    return x_ZF, x_S, res.fun


### Sphere decoding

In [178]:
def sphere_decoding_onebit(y, H, sigma, constellation, rho=None):
    """
    One-bit ADC sphere-decoding detection:
      1. Find the stationary point x_S (e.g. minimizing f(x) via BFGS).
      2. Compute the Hessian at x_S.
      3. Form the quadratic approximation f(x) ≈ f(x_S) + ½ (x-x_S)^T Hess (x-x_S).
         This is equivalent to solving: minimize ||A (x - x_S)||^2,
         where A is the Cholesky factor of Hess (i.e., Hess = A A^T).
      4. Use sphere decoding (for LS problems) to solve
         min_{x in constellation^(n_t)} || A x - A x_S ||^2.
      5. Return the decoded vector.
    """
     
    n_t = H.shape[1]
   
    # Find stationary point x_S, and the ZF point.
    x_zf, x_S, f_val = """"""
    
    # Compute Hessian at x_S.
    Hess = hess_f(x_S, H, y, sigma)
    # Compute Cholesky factor A: Hess = A A^T. (A is lower-triangular.)
    try:
        A = cholesky(Hess)
    except np.linalg.LinAlgError:
        # If Hess is not positive definite, add a small diagonal regularization.
        A = cholesky(Hess + 1e-6*np.eye(n_t))
    # Form the LS problem: minimize || A x - (A x_S) ||^2.
    s = A @ x_S    
      
    # If no initial radius is provided, initialize using the ZF detector.
    if rho is None:
        rho = """"""
        
    # Use sphere decoding for LS problem.
    x_SD, metric = """"""
    
    return x_SD, metric

# Run the simulation

Run a similar simulation as in the conventional MIMO case